In this notebook we want to check how the initial values of mu can affect the optimization

In [253]:
# optimization algorithm to minimize unconstrained function
from scipy.optimize import newton
from math import sqrt
import pandas as pd

import numpy as np
import matplotlib as plt

from tqdm.notebook import tqdm

import plotly.express as px
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Ex 1

In [254]:
# function to minimize
f = lambda x: x[0]**2 + 4*x[1]**2 + 16*x[2]**2
# its gradient/jacobian
JacF = lambda x: [2*x[0], 8*x[1], 32*x[2]]
# norm of its gradient/jacobian
normJacF = lambda x: np.linalg.norm(JacF(x))
# constraint
h = lambda x: x[0] * x[1] - 1

# feasibility penalization function
p = lambda x: 0.5 * h(x)**2
# Merit function
P = lambda mu: lambda x: f(x) + mu*p(x)
JacP = lambda mu: lambda x: np.array([2*x[0] + mu*(x[0]*x[1]**2 -x[1]), 8*x[1] + mu*(x[1]*x[0]**2 -x[0]), 32*x[2]])

Optimization

In [271]:
# initial conditions
x_start = np.array([20,30,15])
x_min1 = np.array((sqrt(2),1/sqrt(2),0))
mu_start_array = [0.5,2,3,5,10,20,50,100,200,500,1000,5000,100000]
                             
# let's create a dataframe with the first 3, 
# then 5th then 10th, then 20th iteration of each optimization

data = pd.DataFrame(columns=['iteration','x', 'f(x)',"||f'(x)||",'P(x)','p(x)','h(x)','mu', 'mu*h(x)','mu_start', 'log10||x_k-x_min||'])
j = 0
for mu_start in tqdm(mu_start_array):
    internal_method_data = pd.DataFrame(columns=['iteration','x', 'f(x)',"||f'(x)||",'P(x)','p(x)','h(x)','mu', 'mu*h(x)','log10||x_k-x_min||']).set_index('iteration') 
    i=0 # iteration
    x = x_start
    mu = mu_start
    mu_update_coef = 2

    # start optimization loop
    max_iter = 200
    while i <= max_iter:
        internal_method_data.loc[i] = [x, f(x), normJacF(x), P(mu)(x), p(x), h(x), mu, mu*h(x), np.log10(np.linalg.norm(x-x_min1))]
        if i in [1,3,5,10,12,15,17,20,50,100,150,200]:
            data.loc[j] = [i, x, f(x), normJacF(x), P(mu)(x), p(x), h(x), mu, mu*h(x),mu_start,np.log10(np.linalg.norm(x-x_min1))]
            j += 1
        i += 1 
        x = newton(JacP(mu), x, maxiter=5000)
        mu += mu_update_coef


/Users/simaonovais/.pyenv/versions/3.8.1/envs/optimization/lib/python3.8/site-packages/scipy/optimize/zeros.py:463: RuntimeWarning:

some failed to converge after 5000 iterations



# Preprocess data

In [275]:
layout = dict(title_text='Convergence of Pure penalization using different starting points for mu', title_x=0.5, xaxis_title='iterations', yaxis_title='log10||x_k-x_min||')
fig = px.line(data, x='iteration',y='log10||x_k-x_min||', color='mu_start')
fig.update_layout(layout)
fig.write_image('media/experiment_1_initial_mu_changing_pure_method_exercise1.png')
fig.show()

In [277]:
layout = dict(title_text='behaviour of mu*h(x) of Pure penalization using different starting points for mu', title_x=0.5, xaxis_title='iterations', yaxis_title='log10||x_k-x_min||')
fig = px.line(data, x='iteration',y='mu*h(x)', color='mu_start')
fig.update_layout(layout)
fig.write_image('media/experiment_1_initial_mu_changing_pure_method_exercise1_muh_behaviour.png')
fig.show()